In [1]:
import pandas as pd
import numpy as np
import dash
import dash_core_components as dcc
import dash_bootstrap_components as dbc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objs as go

In [2]:
competition_matches = pd.read_csv('competition_matches.csv', sep=';')
competition_scores = pd.read_csv('competition_scores.csv', sep=';')
teams = pd.read_csv('teams.csv', sep=';')

competition_scores['created_at'] = pd.to_datetime(competition_scores['created_at'], format='%Y-%m-%d')
competition_scores = competition_scores.drop(['id', 'room_id', 'player_id', 'bracket_no', 'minute'], axis=1)
competition_scores = competition_scores.groupby(['created_at', 'team_id', 'match_id']).sum().reset_index()
competition_scores = competition_scores.drop(['created_at'], axis=1)

match = competition_scores['match_id'].unique()
main = []
for i in match:
    comp = competition_scores[competition_scores['match_id']==i]
    team = comp['team_id'].unique()
    val = np.ones(len(team))
    for num in comp['team_id']:
        for j in range(0,len(team)):
            if num == team[j]:
                main.append(val[j])
                val[j] += 1
competition_scores['main'] = main

teams = teams[['id', 'name']]
df = pd.merge(left=competition_scores, right=teams, left_on='team_id', right_on='id').drop(['id'],axis=1)
competition_matches = competition_matches[['id', 'match_name']]
df = pd.merge(left=df, right=competition_matches, left_on='match_id', right_on='id').drop(['id'],axis=1)
df = df.drop(['team_id', 'match_id'], axis=1)

df['acc_pass'] = (df['pass']/(df['pass']+df['loss']))*100
df['acc_shot'] = (df['on_target']/(df['on_target']+df['off_target']))*100
df['big_chance'] = ((df['goal'] - df['chance_created'] - df['assist'])/(df['free_kick'] + df['penalty_kick'] + df['Corner']))*10
df['kerja_sama'] = (df['goal']/(df['interception'] + df['chance_created'] + df['assist']))*10
df = df[['match_name', 'name', 'main', 'foul', 'yellow', 'red', 'acc_pass', 'acc_shot', 'big_chance', 'kerja_sama', 'goal', 'save']]

df['acc_pass'] = round(df['acc_pass'])
df['acc_shot'] = round(df['acc_shot'])
df['big_chance'] = round(df['big_chance'])
df['kerja_sama'] = round(df['kerja_sama'])

def conv2num(x):
    if x<=0:
        x = 0
        return x
    elif x>100:
        x = 0
        return x
    else:
        return x
a = df['big_chance'].apply(conv2num)
b = df['kerja_sama'].apply(conv2num)
df['big_chance'] = a
df['kerja_sama'] = b
df = df.fillna(0)



In [3]:
m = []
n = []
for j in df['match_name'].unique():
    dff = df[df['match_name']==j]
    n.append(dff['name'].unique())
    m.append(j)

all_options = {}
for i in range(0,len(m)):
    all_options[m[i]] = n[i]

In [4]:
all_max = {}
for i in df['match_name'].unique():
    all_max[i] = {}
    dff = df[df['match_name']==i]
    for j in dff['name'].unique():
        dff = df[df['match_name']==i]
        dff = dff[dff['name']==j]
        v = dff['main'].max()
        all_max[i][j] = v

In [ ]:
app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP, 'https://codepen.io/chriddyp/pen/bWLwgP.css'])
app.layout = html.Div([
    
    html.Br(),
    dbc.Row([
        dbc.Col([
            html.P('Filter by Competition'),
            dcc.Dropdown(
                id = 'select_competition',
                options = [{'label': i, 'value': i} for i in all_options.keys()],
                value = 'Techno Sport Cup'
            ),
            html.Br(),
            html.P('Filter by Team'),
            dcc.Dropdown(
                id = 'select_team'
            ),
            html.Br(),
            html.P('Filter by Match'),
            dcc.Slider(
                id = 'select_main',
                min = 1
            )
        ]),
        dbc.Col([ 
            dbc.Row([
                dbc.Col([
                    html.Div([
                        html.P('ACC PASS'),
                        html.H4(id='num_acc_pass')
                    ])
                ]),
                dbc.Col([
                    html.Div([
                        html.P('COUNT SAVE'),
                        html.H4(id='num_save')
                    ])
                ])
            ]),
            dbc.Row([
                html.Br(),
                dbc.Col([
                    html.Div([
                        html.P('ACC SHOT'),
                        html.H4(id='num_acc_shot')
                    ])
                ]),
                dbc.Col([
                    html.Div([
                        html.P('BIG CHANCE'),
                        html.H4(id='num_big_chance')
                    ])
                ])
            ]),
            dbc.Row([
                html.Br(),
                dbc.Col([
                    html.Div([
                        html.P('KERJA SAMA'),
                        html.H4(id='num_kerja_sama')
                    ])
                ]),
                dbc.Col([
                    html.Div([
                        html.P('COUNT GOAL'),
                        html.H4(id='num_goal')
                    ])
                ])
            ])
        ])
    ]),
    html.Div(id='graph_data', children=dcc.Graph(id='dummy'))
])

# Make Menu Dropdown
@app.callback(
    Output('select_team', 'options'),
    [Input('select_competition', 'value')]
)
def update_dropdown_options(select_competition):
    options = [{'label': i, 'value': i} for i in all_options[select_competition]]
    return options

@app.callback(
    Output('select_team', 'value'),
    [Input('select_team', 'options')]
)
def update_dropdown_value(options):
    return options[0]['value']


# Make Menu Slider
@app.callback(
    Output('select_main', 'max'),
    [Input('select_competition', 'value'),
    Input('select_team', 'value')]
)
def update_slider_max(select_competition, select_team):
    return all_max[select_competition][select_team]

@app.callback(
    Output('select_main', 'value'),
    [Input('select_main', 'max')]
)
def update_slider_value(max):
    return max

@app.callback(
    Output('select_main', 'marks'),
    [Input('select_main', 'min'),
    Input('select_main', 'max')]
)
def update_slider_marks(min, max):
    return {str(num): str(num) for num in range(min, max+1)}

# Helper functions
def match_comp(df, select_competition):
    df = df[df['match_name']==select_competition]
    return df

def name_team(df, select_team):
    df = df[df['name']==select_team]
    return df

def main_team(df, select_main):
    df = df[df['main']==select_main]
    return df


# Make Children Input
# acc_pass
@app.callback(
    Output('num_acc_pass', 'children'),
    [Input('select_competition', 'value'),
    Input('select_team', 'value'),
    Input('select_main', 'value')]
)
def update_num_acc_pass(select_competition, select_team, select_main):
    dff = match_comp(df, select_competition)
    dff = name_team(dff, select_team)
    dff = main_team(dff, select_main)
    dff = dff.reset_index(drop=True)
    return dff['acc_pass'][0]

# save
@app.callback(
    Output('num_save', 'children'),
    [Input('select_competition', 'value'),
    Input('select_team', 'value'),
    Input('select_main', 'value')]
)
def update_num_acc_pass(select_competition, select_team, select_main):
    dff = match_comp(df, select_competition)
    dff = name_team(dff, select_team)
    dff = main_team(dff, select_main)
    dff = dff.reset_index(drop=True)
    return dff['save'][0]

# num_acc_shot
@app.callback(
    Output('num_acc_shot', 'children'),
    [Input('select_competition', 'value'),
    Input('select_team', 'value'),
    Input('select_main', 'value')]
)
def update_num_acc_pass(select_competition, select_team, select_main):
    dff = match_comp(df, select_competition)
    dff = name_team(dff, select_team)
    dff = main_team(dff, select_main)
    dff = dff.reset_index(drop=True)
    return dff['acc_shot'][0]

# big_chance
@app.callback(
    Output('num_big_chance', 'children'),
    [Input('select_competition', 'value'),
    Input('select_team', 'value'),
    Input('select_main', 'value')]
)
def update_num_acc_pass(select_competition, select_team, select_main):
    dff = match_comp(df, select_competition)
    dff = name_team(dff, select_team)
    dff = main_team(dff, select_main)
    dff = dff.reset_index(drop=True)
    return dff['big_chance'][0]

# num_kerja_sama
@app.callback(
    Output('num_kerja_sama', 'children'),
    [Input('select_competition', 'value'),
    Input('select_team', 'value'),
    Input('select_main', 'value')]
)
def update_num_acc_pass(select_competition, select_team, select_main):
    dff = match_comp(df, select_competition)
    dff = name_team(dff, select_team)
    dff = main_team(dff, select_main)
    dff = dff.reset_index(drop=True)
    return dff['kerja_sama'][0]

# num_goal
@app.callback(
    Output('num_goal', 'children'),
    [Input('select_competition', 'value'),
    Input('select_team', 'value'),
    Input('select_main', 'value')]
)
def update_num_acc_pass(select_competition, select_team, select_main):
    dff = match_comp(df, select_competition)
    dff = name_team(dff, select_team)
    dff = main_team(dff, select_main)
    dff = dff.reset_index(drop=True)
    return dff['goal'][0]

#graph_data
@app.callback(
    Output('graph_data', 'children'),
    [Input('select_competition', 'value'),
    Input('select_team', 'value'),
    Input('select_main', 'value')]
)
def update_graph_data(select_competition, select_team, select_main):
    dff = match_comp(df, select_competition)
    dff = name_team(dff, select_team)
    dff = main_team(dff, select_main)
    dff = dff.reset_index(drop=True)
    dff = dff.drop(['match_name', 'name', 'main'], axis=1)
    xx = []
    yy = []
    for i in dff.columns:
        xx.append(i)
        yy.append(dff[i][0])
    image = dcc.Graph(
        figure=go.Figure(
            data = [
                go.Bar(x=xx, y=yy, name='total')
            ],
            layout = go.Layout(title = select_team)
        ),
        id='my-graph'
    )
    return image


if __name__ == '__main__':
    app.run_server(port = 4010)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:4010/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Apr/2020 12:13:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Apr/2020 12:13:39] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [21/Apr/2020 12:13:40] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [21/Apr/2020 12:13:40] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Apr/2020 12:13:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Apr/2020 12:13:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Apr/2020 12:13:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Apr/2020 12:13:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Apr/2020 12:13:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Apr/2020 12:13:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Apr/2020 12:13:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Apr/2020 12:13:43] "POST /_dash-update-component

In [6]:
df

,match_name,name,main,foul,yellow,red,acc_pass,acc_shot,big_chance,kerja_sama,goal,save
0,Techno Sport Cup,WAJAR FC,1.0,2,0,0,0.0,68.0,5.0,12.0,6,9
1,Techno Sport Cup,WAJAR FC,2.0,0,0,0,0.0,75.0,0.0,0.0,0,0
2,Techno Sport Cup,WAJAR FC,3.0,1,0,0,0.0,60.0,0.0,0.0,0,7
3,Techno Sport Cup,Wajar II,1.0,2,0,1,0.0,43.0,10.0,0.0,3,3
4,Techno Sport Cup,Wajar II,2.0,0,0,0,0.0,90.0,0.0,10.0,1,0
5,Techno Sport Cup,Apache,1.0,2,0,0,100.0,76.0,30.0,0.0,3,0
6,Techno Sport Cup,Apache,2.0,1,0,0,0.0,47.0,0.0,0.0,1,0
7,Techno Sport Cup,Apache,3.0,0,0,0,0.0,57.0,0.0,0.0,4,0
8,Techno Sport Cup,YBJ,1.0,3,1,0,0.0,52.0,10.0,30.0,3,13
9,Techno Sport Cup,YBJ,2.0,3,3,1,0.0,67.0,10.0,0.0,1,15
